In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# a) Import required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

import os

In [ ]:
# ==========================================================
# Step 1: Load the Pre-trained CNN Model (VGG16 without top)
# ==========================================================
base_model = VGG16(weights='/content/drive/MyDrive/colab datasets/LP4_datasets/vgg16_weights.h5',
                   include_top=False,
                   input_shape=(224, 224, 3))

print("✅ Pre-trained VGG16 model loaded successfully.")

✅ Pre-trained VGG16 model loaded successfully.


In [ ]:
# ==========================================================
# Step 2: Freeze lower convolutional layers
# ==========================================================
for layer in base_model.layers:
    layer.trainable = False

print("✅ Base layers frozen.")

# ==========================================================
# Step 3: Add custom classifier (fully connected layers)
# ==========================================================
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])


✅ Base layers frozen.


In [ ]:
# ==========================================================
# Step 4: Compile the model
# ==========================================================
model.compile(optimizer=Adam(learning_rate=0.1),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# ==========================================================
# Step 5: Prepare the dataset (images in subfolders)
# ==========================================================
data_dir = "/content/drive/MyDrive/IT Notes/SEM_7_NOTES/LP-4/objects"  # your dataset folder

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% training, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print("✅ Dataset loaded successfully.")


print("Number of classes:", train_generator.num_classes)


Found 446 images belonging to 10 classes.
Found 105 images belonging to 10 classes.
✅ Dataset loaded successfully.
Number of classes: 10


In [ ]:
# ==========================================================
# Step 6: Train the classifier layers
# ==========================================================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 417s 29s/step - accuracy: 0.0940 - loss: 1188.5477 - val_accuracy: 0.0952 - val_loss: 8.2001
Epoch 2/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 369s 27s/step - accuracy: 0.1120 - loss: 22.5461 - val_accuracy: 0.1524 - val_loss: 2.2389
Epoch 3/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 359s 26s/step - accuracy: 0.1582 - loss: 11.0357 - val_accuracy: 0.1619 - val_loss: 2.2499
Epoch 4/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 376s 25s/step - accuracy: 0.1640 - loss: 8.6762 - val_accuracy: 0.1524 - val_loss: 2.2682
Epoch 5/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 352s 25s/step - accuracy: 0.1681 - loss: 2.2581 - val_accuracy: 0.1524 - val_loss: 2.2642


In [ ]:
# ==========================================================
# Step 7: Fine-tune (unfreeze top layers)
# ==========================================================
# Unfreeze last 4 convolutional layers
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Re-compile with smaller learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("🔄 Fine-tuning top layers...")

fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

🔄 Fine-tuning top layers...
Epoch 1/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 414s 30s/step - accuracy: 0.1335 - loss: 2.2795 - val_accuracy: 0.1524 - val_loss: 2.2642
Epoch 2/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 425s 31s/step - accuracy: 0.1686 - loss: 2.2629 - val_accuracy: 0.1524 - val_loss: 2.2642
Epoch 3/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 408s 30s/step - accuracy: 0.1628 - loss: 2.2503 - val_accuracy: 0.1524 - val_loss: 2.2642
Epoch 4/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 424s 31s/step - accuracy: 0.1592 - loss: 9.0043 - val_accuracy: 0.1524 - val_loss: 2.2642
Epoch 5/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 427s 30s/step - accuracy: 0.1471 - loss: 34.7250 - val_accuracy: 0.1524 - val_loss: 2.2642


In [ ]:
# ==========================================================
# Step 8: Evaluate and Save the model
# ==========================================================
loss, acc = model.evaluate(val_generator)
print(f"\n✅ Final Validation Accuracy: {acc*100:.2f}%")

model.save("vgg16_transfer_learning_final.h5")
print("✅ Model saved successfully as vgg16_transfer_learning_final.h5")


4/4 ━━━━━━━━━━━━━━━━━━━━ 66s 15s/step - accuracy: 0.1266 - loss: 2.2781



✅ Final Validation Accuracy: 15.24%
✅ Model saved successfully as vgg16_transfer_learning_final.h5
